In [1]:
#!pip install numpy
#!pip install sklearn
#!python -m pip install nltk
%reset -f
import csv
import numpy
import os
import re
#import nltk
!python --version
#!python


Python 3.9.12


In [2]:
# Load competency questions into pilots
pilots = {}
with open('./data/competency-questions.csv', newline='') as csvcq:
    reader = csv.reader(csvcq, delimiter=',', quotechar='"')
    for row in reader:
        #print(row)
        if row[1] == "" or row[0] == 'text':
            continue;
        if row[1] not in pilots.keys():
            pilots[row[1]] = []
        pilots[row[1]].append(row[0])
#print(pilots)


In [3]:
# Write a file for each pilots with their CQs
for pilot in pilots.keys():
    filename = "./data/docs/" + pilot + ".txt"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    f = open(filename, "w")
    for element in pilots[pilot]:
        f.write(element + "\n")
    f.close()



In [4]:
# Prepare documents
# https://stackoverflow.com/questions/32957895/wordnetlemmatizer-not-returning-the-right-lemma-unless-pos-is-explicit-python
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

lemmatizer = WordNetLemmatizer()
pilots_keys = pilots.keys()
documents = []
for pilot in pilots_keys:
    doc = "\n".join(pilots[pilot])
    doc = re.sub('CQ[0-9]+[^a-zA-Z]', '', doc)
    lemmas = []
    for word, tag in pos_tag(word_tokenize(doc)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
                lemma = word
        else:
                lemma = lemmatizer.lemmatize(word, wntag)
        lemmas.append(lemma)
    doc = " ".join(lemmas)
    word_list = nltk.word_tokenize(doc)
    #print(word_list)
    doc = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    documents.append(doc)
    #print(pilot, doc)

#print(pilots_keys)
#print(documents)


In [5]:
# Compute TF/IDF on each pilot document
from sklearn.feature_extraction.text import TfidfVectorizer
def get_tfidf_top_features(documents,n_top=10):
  tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=None, stop_words='english')
  tfidf = tfidf_vectorizer.fit_transform(documents)
  importance = numpy.argsort(numpy.asarray(tfidf.sum(axis=0)).ravel())[::-1]
  tfidf_feature_names = numpy.array(tfidf_vectorizer.get_feature_names())
  return tfidf_feature_names[importance[:n_top]]

keywords = get_tfidf_top_features(documents, n_top=10)
print(keywords)

['composition' 'source' 'musical' 'music' 'electronic' 'piece' 'play'
 'work' 'perform' 'feature']


/Users/ed4565/Development/polifonia/wp4-competency-questions/.python3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
# https://www.freecodecamp.org/news/how-to-extract-keywords-from-text-with-tf-idf-and-pythons-scikit-learn-b2a0f3d7e667/

from sklearn.feature_extraction.text import TfidfVectorizer

def sort_items(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn(names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    for idx, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(names[idx])
    results={}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=None, stop_words='english')
tfidf_vectorizer.fit(documents)
feature_names = tfidf_vectorizer.get_feature_names()
c = 0
n_top=10
for pilot in pilots_keys:
  doc = documents[c]
  #print(pilot, doc)
  tfidf = tfidf_vectorizer.transform([doc])
  c = c + 1
  sorted_items = sort_items(tfidf.tocoo())
  keywords = extract_topn(feature_names, sorted_items,100)
  print(pilot, keywords)

MEETUPS {'music': 0.412, 'instrument': 0.295, 'musician': 0.258, 'place': 0.196, 'piece': 0.196, 'play': 0.172, 'time': 0.124, 'change': 0.124, 'visit': 0.116, 'sell': 0.116, 'sale': 0.116, 'react': 0.116, 'provenance': 0.116, 'payslip': 0.116, 'patron': 0.116, 'participant': 0.116, 'network': 0.116, 'muciain': 0.116, 'money': 0.116, 'meeting': 0.116, 'listen': 0.116, 'institution': 0.116, 'increase': 0.116, 'happend': 0.116, 'happen': 0.116, 'elite': 0.116, 'did': 0.116, 'come': 0.116, 'cause': 0.116, 'career': 0.116, 'brass': 0.116, 'band': 0.116, 'author': 0.116, 'attendee': 0.116, 'source': 0.113, 'world': 0.098, 'teach': 0.098, 'relationship': 0.098, 'pitch': 0.098, 'number': 0.098, 'meet': 0.098, 'make': 0.098, 'information': 0.098, 'exist': 0.098, 'concert': 0.098, 'account': 0.098, 'relation': 0.086, 'performer': 0.086, 'different': 0.086, 'country': 0.086, 'characterize': 0.086, 'people': 0.076, 'origin': 0.076, 'context': 0.076, 'connect': 0.076, 'use': 0.069, 'event': 0.069,